## 对 Pandas 的简单认识

Pandas 是建立在 Numpy 基础之上建立的新程序库，常与NumPy和Matplotlib一同使用。其核心提供了一种高效的 DataFrame 数据结构。numpy只能处理数值，pandas除了数值，还可以处理其他类型的数值，为用户提供高性能易用数据类型和分析工具

其主要的类为：

*  `pd.Series` 对象，一维，带标签数组
*  `pd.DataFrame` 对象，二维，Series容器
*  `pd.Index` 对象

In [15]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all' # 显示代码块中的所有输出变量

| NumPy              | Pandas             |
| :------------------: | :------------------: |
| 基础数据类型       | 扩展数据类型       |
| 关注数据的结构表达 | 关注数据的应用表达 |
| 维度：数据间的关系 | 数据与索引间的关系 |

### Pandas 中的 nan

numpy中的nan是float类型，但Pandas会自动根据数据类型更改series中的dtype类型

nan可以在数据计算的过程中得到

In [1]:
import pandas as pd
a = pd.Series([1, 2, 3], ['c','d','e'])
b = pd.Series([9, 8, 7, 6], ['a','b','c','d'])
a + b

a    NaN
b    NaN
c    8.0
d    8.0
e    NaN
dtype: float64

也可以手动添加

In [53]:
data = pd.DataFrame(np.arange(28).reshape(7,4),index=list("ABCDEFG"),columns=list("WXYZ"))
data.loc[['E','F'],['W','X']] = np.nan
data

,W,X,Y,Z
A,0.0,1.0,2,3
B,4.0,5.0,6,7
C,8.0,9.0,10,11
D,12.0,13.0,14,15
E,NaN,NaN,18,19
F,NaN,NaN,22,23
G,24.0,25.0,26,27


关于显示nan的函数

In [55]:
print(pd.isnull(data),'\n')
print(pd.notnull(data),'\n')
print(data[pd.notnull(data["W"])],'\n')
print(pd.notnull(data["W"]))

       W      X      Y      Z
A  False  False  False  False
B  False  False  False  False
C  False  False  False  False
D  False  False  False  False
E   True   True  False  False
F   True   True  False  False
G  False  False  False  False 

       W      X     Y     Z
A   True   True  True  True
B   True   True  True  True
C   True   True  True  True
D   True   True  True  True
E  False  False  True  True
F  False  False  True  True
G   True   True  True  True 

      W     X   Y   Z
A   0.0   1.0   2   3
B   4.0   5.0   6   7
C   8.0   9.0  10  11
D  12.0  13.0  14  15
G  24.0  25.0  26  27 

A     True
B     True
C     True
D     True
E    False
F    False
G     True
Name: W, dtype: bool


In [56]:
print(data.dropna(axis=0))
# 不显示有NaN的行
print(data.dropna(axis=0,how="all"))
# 不显示全为NaN的行，参数how指定
print(data.dropna(axis=0,how="any"))
# 不显示有一个为NaN的行
print(data.dropna(axis=1,how="any"))

# data.dropna(axis=0,inplace=True) <==> data=data.dropna(axis=0)

      W     X   Y   Z
A   0.0   1.0   2   3
B   4.0   5.0   6   7
C   8.0   9.0  10  11
D  12.0  13.0  14  15
G  24.0  25.0  26  27
      W     X   Y   Z
A   0.0   1.0   2   3
B   4.0   5.0   6   7
C   8.0   9.0  10  11
D  12.0  13.0  14  15
E   NaN   NaN  18  19
F   NaN   NaN  22  23
G  24.0  25.0  26  27
      W     X   Y   Z
A   0.0   1.0   2   3
B   4.0   5.0   6   7
C   8.0   9.0  10  11
D  12.0  13.0  14  15
G  24.0  25.0  26  27
    Y   Z
A   2   3
B   6   7
C  10  11
D  14  15
E  18  19
F  22  23
G  26  27


填补空值

In [57]:
print(data)
n = 0
print(data.fillna(n))
print(data.mean())
# 计算均值，而且在Pandas中会直接忽略NaN，而不是返回nan
print(data["W"].fillna(data["W"].mean()))

      W     X   Y   Z
A   0.0   1.0   2   3
B   4.0   5.0   6   7
C   8.0   9.0  10  11
D  12.0  13.0  14  15
E   NaN   NaN  18  19
F   NaN   NaN  22  23
G  24.0  25.0  26  27
      W     X   Y   Z
A   0.0   1.0   2   3
B   4.0   5.0   6   7
C   8.0   9.0  10  11
D  12.0  13.0  14  15
E   0.0   0.0  18  19
F   0.0   0.0  22  23
G  24.0  25.0  26  27
W     9.6
X    10.6
Y    14.0
Z    15.0
dtype: float64
A     0.0
B     4.0
C     8.0
D    12.0
E     9.6
F     9.6
G    24.0
Name: W, dtype: float64


### 初试

In [41]:
import numpy as np
import pandas as pd
data = pd.Series(np.arange(10))
print(data)
print(data.cumsum()) # 计算前N项累加和

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
dtype: int32
0     0
1     1
2     3
3     6
4    10
5    15
6    21
7    28
8    36
9    45
dtype: int32


## Pandas 的 Series 类

Series 对象是一个**带索引数据**构成的**一维数组**, 它除了存储数据外，还**显式**地把**索引**也存储下来！

Numpy 数据用的是**隐式**的索引！

Series 对象由一组数据及与之相关的数据索引组成：

| index_0 | data_a |
| :-----: | :----: |
| index_1 | data_b |
| index_2 | data_c |
| index_3 | data_d |

In [9]:
test_0 = pd.Series([9, 8, 7, 6])
print(test_0)
# 前一列是自动生成的索引，称为自动索引，也就是标签，默认从零开始递增，也可以用 **index参数** 指定，但数目要与数据量相同
# dtype: int64    是 NumPy 中的数据类型，以源数据的形式显示

0    9
1    8
2    7
3    6
dtype: int64


In [4]:
test_1 = pd.Series([9, 8, 7, 6], index=['a','b','c','d'])
print(test_1)
# 同时也可以通过指定index参数来实现自定义索引。如果index参数放在第二个参数位置，其实也可以省略'index='

a    9
b    8
c    7
d    6
dtype: int64


### 属性

Series本质上由两个数组组成，一个构成了对象的键（index， 索引），另一个构成了对象的值（values），键 -> 值
    * ndarray的很多方法都使用于Series类型，比如argmax，clip
    * Series也具有where方法，但结果和ndarray不同
    * Series对象可以随时修改并即刻生效

这也就是 Series 对象最主要的两个基本属性： 

* `.index`    因此可迭代
    * len(test.index)可以获取Series的长度
    * 还可以对其使用 list()
* `.values`   形同字典
    * 遍历
    * 切片
* `.name`
    * Series对象和索引都可以有一个名字，存储在属性 `.name` 中

In [13]:
data = pd.Series(np.arange(5))
print(data,'\n')
print(data.values,'\n')
print(type(data.values),'\n')
print(data.index,'\n')
print(type(data.index),'\n')

0    0
1    1
2    2
3    3
4    4
dtype: int32 

[0 1 2 3 4] 

<class 'numpy.ndarray'> 

RangeIndex(start=0, stop=5, step=1) 

<class 'pandas.core.indexes.range.RangeIndex'> 



In [20]:
data.name = "Series对象"
data.index.name = "索引列"
print(data)

索引列
a    0
b    1
c    2
d    3
e    4
Name: Series对象, dtype: int32


### 创建

Series类型可以由python列表、标量值、python字典、ndarray 或 其他函数来创建

In [10]:
test_2 = pd.Series(25, index=['a','b','c'])       # 通过标量创建Series类型，此时 index= 不能省略
print(test_2)
print(type(test_2))

a    25
b    25
c    25
dtype: int64
<class 'pandas.core.series.Series'>


In [29]:
test_3 = pd.Series({'a':9, 'b':8, 'c':7})       # 通过字典创建Series类型
print(test_3)
test_4 = pd.Series({'a':9, 'b':8, 'c':7}, index=['c','a','b','d'])
print(test_4)
# 字典类型是无序的，所以index参数从字典中进行选择操作，可以来改变字典中数据的顺序。
# `index` 参数还有另外的用处，可以用来筛选字典中的键值对，即选择字典中的一部分元素来构建 Series 对象 。

test_4 = pd.Series({'a':9, 'b':8, 'c':7}, index=['a','c'])
print(test_4)

a    9
b    8
c    7
dtype: int64
c    7.0
a    9.0
b    8.0
d    NaN
dtype: float64
a    9
c    7
dtype: int64


In [8]:
test_5 = pd.Series(np.arange(4))       # 通过array类型创建Series类型
print(test_5) 
test_6 = pd.Series(np.arange(4), index=np.arange(9,5,-1)) # index参数也可以通过array类型来指定值
print(test_6)

0    0
1    1
2    2
3    3
dtype: int32
9    0
8    1
7    2
6    3
dtype: int32


### 操作

Series 是通用的 Numpy 一维数组， 但比 Numpy 更加灵活。<br>
用户可以用与 Numpy 数组类似的索引方式索引。

In [34]:
data = pd.Series(np.arange(5),index=['a', 'b', 'c', 'd', 'e'])
print(data,'\n')
print(data['a'],'\n')
print(data[0],'\n')            # 自动索引是默认生成的，即使自定义了索引，也可以通过自动索引来索引数据
print(data['a':'c'],'\n')      
print('a' in data)
print(data[(data > 2) & (data <= 5)]) # 布尔索引(或者叫掩码)
print(data[['a', 'e']]) # 花式索引

a    0
b    1
c    2
d    3
e    4
dtype: int32 

0 

0 

a    0
b    1
c    2
dtype: int32 

True


In [30]:
print(data.keys())
print(list(data.items()))

Index(['a', 'b', 'c', 'd', 'e'], dtype='object')
[('a', 0), ('b', 1), ('c', 2), ('d', 3), ('e', 4)]



* 索引可用于切片操作，如果自定义索引也是同自动索引一样的非负整数，则优先当作自动索引来处理结果。
    * 这种情况被称为隐式索引，而其他的直接使用索引的情况成为显式索引
    * 这样有时会引起混乱，所以还有loc，iloc等索引器可用来进行索引
    * <span class="mark">对于 Series 对象，loc指定使用显式索引，iloc指定使用隐式索引</span>
* 但两套索引不能同时使用


* 还有一种ix方法，它更像是loc和iloc两种切片方法的融合。ix方法在使用时既可以接收索引名称也可以接收索引位置。
    * 控制ix方法需要注意以下几点：
        * 使用ix方法时，当索引名称和位置存在部分重叠时，ix默认优先识别名称。
        * 尽量使用列索引名称，而非列索引位置，主要用来保证代码可读性。
        * 使用列索引位置时，需要注解，同样保证代码可读性。
        * 除此之外ix方法还有一个缺点，就是在面对数据量巨大的任务的时候，其效率会低于loc和iloc方法，所以在日常的数据分析工作中建议使用loc和iloc方法来执行切片操作。

In [40]:
data = pd.Series(np.arange(5),index=[1,2,3,4,5])
print(data.loc[1],'\n')
print(data.loc[1:3],'\n')
print(data.iloc[1],'\n')
print(data.iloc[1:3],'\n')

0 

1    0
2    1
3    2
dtype: int32 

1 

2    1
3    2
dtype: int32 



### Series 和其他类型的对比

Series类型与array类型：

* 索引方法相同，采用[]
* NumPy中运算和操作可用于Series类型
* 可以通过自定义索引的列表进行切片
* 可以通过自动索引进行切片，如果存在自定义索引，则一同被切片

Series类型与字典类型：

* 通过自定义索引访问
* 保留字 in 操作
* 使用 .get() 方法

## Pandas 的 DataFarme 类

**DataFrame** 类型的数据，是一个Series类型的容器，由共用相同索引的一组列组成。DataFrame是一个表格型的数据类型，每列值类型可以不同。它既有行索引，列索引。

| 索引 | 多  | 列     | 数   | 据     |
| :--------: | :----------: | :------: | :----: | :------: |
| index  $\downarrow$  (axis=0) | column  $\rightarrow$  (axis=1) ||||
| index_0| data_a | data_1 | ...  | data_w |
| index_1| data_b | data_2 |     | data_x |
| index_2| data_c | data_3 |     | data_y |
| index_3| data_d | data_4 |     | data_z |

In [2]:
import numpy as np
import pandas as pd
pd.DataFrame(np.arange(12).reshape(3,4))

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


* 比Series类型多了两块数据，
    * 上方类似于列号的数据
        * 纵向索引、列索引
        * 叫columns，1轴，axis=1
    * 左面的索引
        * 横向索引、行索引
        * 叫index，0轴，axis=0

In [7]:
test_dataframe_0 = pd.DataFrame(np.arange(12).reshape(3,4),index=list("ABC"),columns=list("WXYZ"))
print('原数据为：\n',test_dataframe_0)
print('\nX列的数据为：\n',test_dataframe_0.X)
print('\nX列的数据为：\n',test_dataframe_0['X'])
print('\nX列A行的数据为：\n',test_dataframe_0.X.A)
print('\nX列A行的数据为：\n',test_dataframe_0['X']['A'])

原数据为：
    W  X   Y   Z
A  0  1   2   3
B  4  5   6   7
C  8  9  10  11

X列的数据为：
 A    1
B    5
C    9
Name: X, dtype: int32

X列的数据为：
 A    1
B    5
C    9
Name: X, dtype: int32

X列A行的数据为：
 1

X列A行的数据为：
 1


### 创建

DataFrame类型可以由如下类型创建：

* 二维array类型对象
* 由一维array、列表、字典、元组 或 Series构成的字典
* Series 类型
* 其他的 DataFrame 类型

In [11]:
pd.DataFrame(np.arange(10).reshape(2,5))

,0,1,2,3,4
0,0,1,2,3,4
1,5,6,7,8,9


In [12]:
dt = {'one':pd.Series([1, 2, 3], index=['a','b','c']),'two':pd.Series([9, 8, 7, 6],index=['a','b','c','d'])}
pd.DataFrame(dt)

,one,two
a,1.0,9
b,2.0,8
c,3.0,7
d,NaN,6


In [13]:
pd.DataFrame(dt, index=['b','c','d'], columns=['two','three'])

,two,three
b,8,NaN
c,7,NaN
d,6,NaN


In [14]:
dl = {'one':[1, 2, 3, 4], 'two':[9, 8, 7, 6]}
pd.DataFrame(dl, index=['a','b','c','d'])

,one,two
a,1,9
b,2,8
c,3,7
d,4,6


### 可由当前列直接在本DataFrame类型变量中添加新列，脱胎于字典操作

In [4]:
dl = {'one':[1, 2, 3, 4], 'two':[9, 8, 7, 6]}
data = pd.DataFrame(dl)
data['three'] = data['one']+data['two']
data

,one,two,three
0,1,9,10
1,2,8,10
2,3,7,10
3,4,6,10


### 属性

dataframe类型的常用属性：

* `.shape`  行数，列数
* `.dtypes`  列数据类型
* `.ndim`  数据维度
* `.index`  行索引
* `.columns`  列索引
* `.values`  对象值，二维ndarray的数组





DataFrame是一个二维带“标签”的数组

In [15]:
test_dataframe_1 = pd.DataFrame(np.arange(12).reshape(3,4),index=list("ABC"),columns=list("WXYZ"))
print('原数据为：\n',test_dataframe_1)
print('\n数据的shape属性为：\n',test_dataframe_1.shape)
print('\n数据的dtypes属性为：\n',test_dataframe_1.dtypes)
print('\n数据的ndim属性为：\n',test_dataframe_1.ndim)
print('\n数据的index属性为：\n',test_dataframe_1.index)
print('\n数据的columns属性为：\n',test_dataframe_1.columns)
print('\n数据的values属性为：\n',test_dataframe_1.values)

原数据为：
    W  X   Y   Z
A  0  1   2   3
B  4  5   6   7
C  8  9  10  11

数据的shape属性为：
 (3, 4)

数据的dtypes属性为：
 W    int32
X    int32
Y    int32
Z    int32
dtype: object

数据的ndim属性为：
 2

数据的index属性为：
 Index(['A', 'B', 'C'], dtype='object')

数据的columns属性为：
 Index(['W', 'X', 'Y', 'Z'], dtype='object')

数据的values属性为：
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


### 部分操作

#### 索引 

In [17]:
data = pd.DataFrame(np.arange(24).reshape(6,4),index=list("ABCEFG"),columns=list("WXYZ"))
data

,W,X,Y,Z
A,0,1,2,3
B,4,5,6,7
C,8,9,10,11
E,12,13,14,15
F,16,17,18,19
G,20,21,22,23


In [24]:
print('第X列的数据为：\n',data['X'])
print('\n第X列的数据为：\n',data.X)
print('\n第2行及其后的数据为（切片操作默认优先切列，即取整行）：\n',data[2:])
print('\nX列第2行及其后的数据为：\n',data[2:]['X'])
print('\nX列第2行及其后的数据为：\n',data['X'][2:])

print('\n如果取出一列，类型为Series，如果取出多列，类型则为DataFrame。\n')
print(type(data[:3]))
print(type(data['X']))

第X列的数据为：
 A     1
B     5
C     9
E    13
F    17
G    21
Name: X, dtype: int32

第X列的数据为：
 A     1
B     5
C     9
E    13
F    17
G    21
Name: X, dtype: int32

第2行及其后的数据为（切片操作默认优先切列，即取整行）：
     W   X   Y   Z
C   8   9  10  11
E  12  13  14  15
F  16  17  18  19
G  20  21  22  23

X列第2行及其后的数据为：
 C     9
E    13
F    17
G    21
Name: X, dtype: int32

X列第2行及其后的数据为：
 C     9
E    13
F    17
G    21
Name: X, dtype: int32

如果取出一列，类型为Series，如果取出多列，类型则为DataFrame。

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


还有其他的一些方法

* df.loc[ ] 通过标签索引行数据
* df.iloc[ ] 通过位置获取行数据

In [37]:
print('A行Z列的数据为：\n',data.loc['A','Z'])
print('\nZ列的数据为：\n',data.loc[:,'Z'])
print('\nA行的数据为：\n',data.loc['A',:])
print('\nA行C行的数据为：\n',data.loc[['A','C'],:])
print('\nX列Z列的数据为：\n',data.loc[:,['X','Z']])
print('\n位于A行C行X列Z列的数据为：\n',data.loc[['A','C'],['X','Z']])

print('\n',data.iloc[0,:],'\n')
print(data.iloc[:,0],'\n')
# 其他操作和loc相似

A行Z列的数据为：
 3

Z列的数据为：
 A     3
B     7
C    11
E    15
F    19
G    23
Name: Z, dtype: int32

A行的数据为：
 W    0
X    1
Y    2
Z    3
Name: A, dtype: int32

A行C行的数据为：
    W  X   Y   Z
A  0  1   2   3
C  8  9  10  11

X列Z列的数据为：
     X   Z
A   1   3
B   5   7
C   9  11
E  13  15
F  17  19
G  21  23

位于A行C行X列Z列的数据为：
    X   Z
A  1   3
C  9  11

 W    0
X    1
Y    2
Z    3
Name: A, dtype: int32 

A     0
B     4
C     8
E    12
F    16
G    20
Name: W, dtype: int32 



##### 布尔索引

In [38]:
data

,W,X,Y,Z
A,0,1,2,3
B,4,5,6,7
C,8,9,10,11
E,12,13,14,15
F,16,17,18,19
G,20,21,22,23


In [39]:
data['X']>15

A    False
B    False
C    False
E    False
F     True
G     True
Name: X, dtype: bool

In [40]:
data[data.X>15]

,W,X,Y,Z
F,16,17,18,19
G,20,21,22,23


In [42]:
data[(data.X>8) & (data.X<20)]

,W,X,Y,Z
C,8,9,10,11
E,12,13,14,15
F,16,17,18,19


#### 整体特征查询

* df.head()    显示头部几行，默认5行
* df.tail()    显示末尾几行，默认5行
* df.info()    相关信息概览：行数、列数、列索引、列非空值个数、列属性、列类型、内存占用
* df.describe() 快速综合统计结果：计数、均值、标准差、最大值、四分位数、最小值

## index(类)

Series 和 DataFrame 的索引是 Index 类型，这也确实是单独一个类，但通常使用的情况不多，一般都是直接改变Series或DataFrame而不是index索引。

index索引类型的常用方法：

| 方法 | 说明 |
| :--------------------: | :----: |
| .append(index)       | 连接另一个 Index 对象，产生新的 Index 对象 |
| .diff(index)         | 计算差集，产生新的 Index 对象              |
| .intersection(index) | 计算交集                                   |
| .union(index)        | 计算并集                                   |
| .delete(loc)         | 删除 loc 位置的元素                        |
| .insert(loc.e)       | 在 loc 位置增加一个元素 e                  |

* 例如：d.index.append()

## 操作

### 索引更改

.reindex() 能够改变或重排 Series 和 DataFrame 索引

.reindex(index=None,columns=None,...) 的参数

* index, columns    新的行列自定义索引  
* fill_value    重新索引中，用于填充缺失位置的值
* method    填充方法，ffill当前值向前填充，bfill向后填充
* limit    最大值填充量
* copy    默认True，生成的对象，False时，新旧相等不复制

In [5]:
dl = {'column0':[1, 2, 3, 4, 5],'column1':[1, 2, 3, 4, 5],'column2':[1, 2, 3, 4, 5],'column3':[1, 2, 3, 4, 5],'column4':[1, 2, 3, 4, 5]}
d = pd.DataFrame(dl, index=['c1','c2','c3','c4','c5'])
d

,column0,column1,column2,column3,column4
c1,1,1,1,1,1
c2,2,2,2,2,2
c3,3,3,3,3,3
c4,4,4,4,4,4
c5,5,5,5,5,5


In [6]:
di = d.reindex(index=['c4','c2','c1','c3','c5'])
di

,column0,column1,column2,column3,column4
c4,4,4,4,4,4
c2,2,2,2,2,2
c1,1,1,1,1,1
c3,3,3,3,3,3
c5,5,5,5,5,5


In [7]:
dc = d.reindex(columns=['column2','column1','column0','column4','column3'])
dc

,column2,column1,column0,column4,column3
c1,1,1,1,1,1
c2,2,2,2,2,2
c3,3,3,3,3,3
c4,4,4,4,4,4
c5,5,5,5,5,5


### DataFrame中排序的方法

* df.sort_values()     默认为升序排序
    * df.sort_values(
    by,
    axis=0,
    ascending=True,
    inplace=False,
    kind='quicksort',
    na_position='last',)
        * by 指定排序源列或行
        * axis 指定排序源来自行还是列
        * ascendin 指定排序方向，默认True为升序排序
        * inplace
        * kind
        * na_position

## 运算与函数

### 算数运算

根据行列索引，补齐后运算，运算默认产生浮点数（补齐时缺项填充NaN空值）。当维度不同的对象之间进行运算时为广播运算

In [11]:
import numpy as np
import pandas as pd
a = pd.DataFrame(np.arange(12).reshape(3, 4))
b = pd.DataFrame(np.arange(20).reshape(4, 5))
a
b
a + b    # NaN与任何值运算结果都是NaN
a - b

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


,0,1,2,3,4
0,0,1,2,3,4
1,5,6,7,8,9
2,10,11,12,13,14
3,15,16,17,18,19


,0,1,2,3,4
0,0.0,2.0,4.0,6.0,NaN
1,9.0,11.0,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


,0,1,2,3,4
0,0.0,0.0,0.0,0.0,NaN
1,-1.0,-1.0,-1.0,-1.0,NaN
2,-2.0,-2.0,-2.0,-2.0,NaN
3,NaN,NaN,NaN,NaN,NaN


数据类型的算术运算方法形式的运算：

| 方法             | 说明                     |
| :----------------: | :------------------------: |
| .add(d, **argws) | 类型间加法运算，可选参数 |
| .sub(d, **argws) | 类型间减法运算，可选参数 |
| .mul(d, **argws) | 类型间乘法运算，可选参数 |
| .div(d, **argws) | 类型间除法运算，可选参数 |

In [12]:
b.add(a, fill_value=100)   # fill_value参数代替NaN，代替后参与运算

,0,1,2,3,4
0,0.0,2.0,4.0,6.0,104.0
1,9.0,11.0,13.0,15.0,109.0
2,18.0,20.0,22.0,24.0,114.0
3,115.0,116.0,117.0,118.0,119.0


In [13]:
a.mul(b, fill_value=0)

,0,1,2,3,4
0,0.0,1.0,4.0,9.0,0.0
1,20.0,30.0,42.0,56.0,0.0
2,80.0,99.0,120.0,143.0,0.0
3,0.0,0.0,0.0,0.0,0.0


In [17]:
c = pd.Series(np.arange(4)) # 广播运算
c
c-10
b
b-c

0    0
1    1
2    2
3    3
dtype: int32

0   -10
1    -9
2    -8
3    -7
dtype: int32

,0,1,2,3,4
0,0,1,2,3,4
1,5,6,7,8,9
2,10,11,12,13,14
3,15,16,17,18,19


,0,1,2,3,4
0,0.0,0.0,0.0,0.0,NaN
1,5.0,5.0,5.0,5.0,NaN
2,10.0,10.0,10.0,10.0,NaN
3,15.0,15.0,15.0,15.0,NaN


### 比较运算

只能比较相同索引的元素，不进行补齐。不同维度间的比较运算为广播运算。采用 >、 <、 >=、 <=、 ==、 != 等符号来进行二元运算产生布尔对象。

In [19]:
a
d = pd.DataFrame(np.arange(12,0,-1).reshape(3, 4)) # 同维度运算要求尺寸一致
d
c

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


,0,1,2,3
0,12,11,10,9
1,8,7,6,5
2,4,3,2,1


0    0
1    1
2    2
3    3
dtype: int32

In [22]:
a > d
a == d
a > c
c > 0

,0,1,2,3
0,False,False,False,False
1,False,False,False,True
2,True,True,True,True


,0,1,2,3
0,False,False,False,False
1,False,False,True,False
2,False,False,False,False


,0,1,2,3
0,False,False,False,False
1,True,True,True,True
2,True,True,True,True


0    False
1     True
2     True
3     True
dtype: bool

### 排序

(Pandas库的数据排序，NaN统一放在排序末尾)

* `.sort_index(axis=0, ascending=True)` 方法在指定轴上根据索引进行排序，默认升序

In [24]:
b = pd.DataFrame(np.arange(20).reshape(4,5), index=['c','a','d','b'])
b
b.sort_index()
b.sort_index(ascending=False)
b.sort_index(ascending=False,axis=1)

,0,1,2,3,4
c,0,1,2,3,4
a,5,6,7,8,9
d,10,11,12,13,14
b,15,16,17,18,19


,0,1,2,3,4
a,5,6,7,8,9
b,15,16,17,18,19
c,0,1,2,3,4
d,10,11,12,13,14


,0,1,2,3,4
d,10,11,12,13,14
c,0,1,2,3,4
b,15,16,17,18,19
a,5,6,7,8,9


,4,3,2,1,0
c,4,3,2,1,0
a,9,8,7,6,5
d,14,13,12,11,10
b,19,18,17,16,15


* `.sort_values()` 方法再指定轴上根据数值进行排序，默认升序
    * Series.sort_values(axis=0, ascending=True)
    * DataFrame.sort_values(by, axis=0, ascending=True)   by：axis轴上的某个索引或索引列表 ，根据这些行或列来排序

In [25]:
a = pd.DataFrame(np.arange(20).reshape(4,5),index=['c','d','a','b'])

In [26]:
a
a.sort_values(2, ascending=False)
a.sort_values('a', axis=1, ascending=False)

,0,1,2,3,4
c,0,1,2,3,4
d,5,6,7,8,9
a,10,11,12,13,14
b,15,16,17,18,19


,0,1,2,3,4
b,15,16,17,18,19
a,10,11,12,13,14
d,5,6,7,8,9
c,0,1,2,3,4


,4,3,2,1,0
c,4,3,2,1,0
d,9,8,7,6,5
a,14,13,12,11,10
b,19,18,17,16,15


### 基本统计分析函数

适用于两种类型：

| 方法                | 说明                             |
| :-------------------: | :--------------------------------: |
| .sum()              | 计算数据的总和，按0轴计算，下同  |
| .count()            | 非NaN值的数量                    |
| .mean()   .median() | 计算数据的算术平均值、算术中位数 |
| .var()   .std()     | 计算数据的方差、标准差           |
| .min()   .max()     | 计算数据的最小值、最大值         |

适用于Series和DataFrame类型的方法还有：.describe() 针对0轴（各列）的统计汇总

适用于Series类型：

| 方法   | 说明  |
| :---------------------: | :-----------------------------------------: |
| .argmin()   .argmax() | 计算数据最大值、最小值所在位置的索引位置（自动索引） |
| .idxmin()   .idxmax() | 计算数据最大值、最小值所在位置的索引（自定义索引）   |

In [38]:
a = pd.Series([9, 8, 7, 6], index=['a','b','c','d'])
b = pd.DataFrame(np.arange(20).reshape((4,5)),index=['c','a','d','b'])
a
a.describe()
type(a.describe())
a.describe()['count']
a.describe()['max']
b
b.describe()
b.describe().loc['max']
b.describe()[2]

a    9
b    8
c    7
d    6
dtype: int64

count    4.000000
mean     7.500000
std      1.290994
min      6.000000
25%      6.750000
50%      7.500000
75%      8.250000
max      9.000000
dtype: float64

pandas.core.series.Series

4.0

9.0

,0,1,2,3,4
c,0,1,2,3,4
a,5,6,7,8,9
d,10,11,12,13,14
b,15,16,17,18,19


,0,1,2,3,4
count,4.000000,4.000000,4.000000,4.000000,4.000000
mean,7.500000,8.500000,9.500000,10.500000,11.500000
std,6.454972,6.454972,6.454972,6.454972,6.454972
min,0.000000,1.000000,2.000000,3.000000,4.000000
25%,3.750000,4.750000,5.750000,6.750000,7.750000
50%,7.500000,8.500000,9.500000,10.500000,11.500000
75%,11.250000,12.250000,13.250000,14.250000,15.250000
max,15.000000,16.000000,17.000000,18.000000,19.000000


0    15.0
1    16.0
2    17.0
3    18.0
4    19.0
Name: max, dtype: float64

count     4.000000
mean      9.500000
std       6.454972
min       2.000000
25%       5.750000
50%       9.500000
75%      13.250000
max      17.000000
Name: 2, dtype: float64

#### 累计统计分析函数

累计统计分析函数：（适用于两种类型）

| 方法       | 说明                               |
| :----------: | :----------------------------------: |
| .cumsum()  | 依次给出前1、2、...、n个数的和     |
| .cumprod() | 依次给出前1、2、...、n个数的积     |
| .cummax()  | 依次给出前1、2、...、n个数的最大值 |
| .cummin()  | 依次给出前1、2、...、n个数的最小值 |

滚动计算（窗口计算）函数：

| 方法                       | 说明                                |
| :--------------------------: | :-----------------------------------: |
| .rolling(w).sum()          | 依次计算相邻w个元素的和             |
| .rolling(w).mean()         | 依次计算相邻w个元素的算术平均值     |
| .rolling(w).var()          | 依次计算相邻w个元素的方差           |
| .rolling(w).min()          | 依次计算相邻w个元素的标准差         |
| .rolling(w).min()   .max() | 依次计算相邻w个元素的最小值和最大值 |

In [39]:
b = pd.DataFrame(np.arange(20).reshape((4,5)),index=['c','a','d','b'])
b
b.rolling(2).sum()
b.rolling(3).sum()

,0,1,2,3,4
c,0,1,2,3,4
a,5,6,7,8,9
d,10,11,12,13,14
b,15,16,17,18,19


,0,1,2,3,4
c,NaN,NaN,NaN,NaN,NaN
a,5.0,7.0,9.0,11.0,13.0
d,15.0,17.0,19.0,21.0,23.0
b,25.0,27.0,29.0,31.0,33.0


,0,1,2,3,4
c,NaN,NaN,NaN,NaN,NaN
a,NaN,NaN,NaN,NaN,NaN
d,15.0,18.0,21.0,24.0,27.0
b,30.0,33.0,36.0,39.0,42.0


#### 相关分析

两个事物，表示WieX和Y，如何判断它们之间的存在相关性（正相关与负相关与不相关）

这就用到了协方差：

$$
cov(X,Y)=\frac{\sum_1^n(X_i-\bar{X})(Y_i-\bar{Y})}{n-1}
$$

* 协方差>0，X和Y正相关
* 协方差<0，X和Y负相关
* 协方差=0，X和Y独立无关

Pearson相关系数

$$
r = \frac{N\sum x_iy_i-\sum x_i \sum y_i}{\sqrt{N\sum x_i^2-(\sum x_i)^2}\sqrt{N\sum{y_i^2}-(\sum{y_i})^2}}
$$

(r 的取值范围为 [-1, 1])

相关分析函数：（适用于两种类型）

| 方法    | 说明  |
| :-------: | :---------: |
| .cov()  | 计算协方差矩阵                                     |
| .corr() | 计算相关系数矩阵，Pearson、Spearman、Kendall等系数 |

In [41]:
hprice = pd.Series([3.04, 22.91, 12.75, 22.6, 12.33],index=['2008','2009','2010','2011','2012'])
m2 = pd.Series([8.18, 18.38, 9.13, 7.82, 6.69],index=['2008','2009','2010','2011','2012'])
hprice.corr(m2)

0.5231958405846526

## 使用本地文件存取数据

### Pandas读取外部数据
* 使用 **pd.read_** ,然后根据外部数据文件的类型指定具体方法
    * read_csv
    * read_excel
    * read_html
    * read_sql
    * 等等

In [ ]:
test_data_002 = pd.read_csv("test_data.csv")
print(test_data)

## 便捷查找